In [ ]:
!pwd

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
train_path = '../input/siim-isic-melanoma-classification/jpeg/train'

In [ ]:
##Exploratory Data Analysis EDA

In [ ]:
df= pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
df.head(2)

In [ ]:
#Edit the image name
#df['jpg']='.jpg'
#df['image_name']= df['image_name']+df['jpg']
#df.head(3)

In [ ]:
#Drop Duplicates

dup = pd.read_csv("../input/melanoma-duplicate/2020_Challenge_duplicates.csv")

drop_idx_list = []
for dup_image in dup.ISIC_id_paired:
    for idx,image in enumerate(df.image_name):
        if image == dup_image:
            drop_idx_list.append(idx)

print("no. of duplicates in training dataset:",len(drop_idx_list))

df.drop(drop_idx_list,inplace=True)

print("updated dimensions of the training dataset:",df.shape)

In [ ]:
df.value_counts(['target'])

In [ ]:
def split(df,train_size,stratify_label,randomstate=42):
    
    from sklearn.model_selection import train_test_split
    
    train,valid=train_test_split(df,train_size=train_size,stratify=df[stratify_label],random_state=randomstate)
    
    positive_index=train[train[stratify_label]==1].index.tolist()
    random_negative_index=train[train[stratify_label]==0].sample(len(positive_index),random_state=randomstate).index.tolist()
    
    balanced_train=train.loc[positive_index+random_negative_index]
    
    return balanced_train,valid

In [ ]:
splitting=split(df,train_size=0.8,stratify_label='target')
train,valid=splitting
for dataframe in splitting:
    print(dataframe['target'].value_counts(),end='\n\n')

In [ ]:
import shutil
import os
train_path = '../input/siim-isic-melanoma-classification/jpeg/train'
model_path = 'model_images'

In [ ]:
def prepare_dirs(directory,dataframe,target_column):
    target_classes=dataframe[target_column].unique().tolist()
    if directory in os.listdir():
        shutil.rmtree(directory)
    os.mkdir(directory)
    for use in 'train','valid':
        os.mkdir(f'{directory}/{use}')
        for x in target_classes:
            os.mkdir(f'{directory}/{use}/{x}')

In [ ]:
prepare_dirs(model_path,df,'target')

In [ ]:
for i,ddf in enumerate(splitting):
    phase={0:'train',1:'valid'}
    phase=phase[i]
    ddf.apply(lambda x: shutil.copy2(f'{train_path}/{x[0]}.jpg',f'{model_path}/{phase}/{x[-1]}'),axis=1)

In [ ]:
#IMPORT LIBRARIES
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models # add models to the list
from torchvision.utils import make_grid
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Define Transformers
train_transform = transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(300),             # resize shortest side to 300 pixels
        transforms.CenterCrop(300),         # crop longest side to 300 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

test_transform = transforms.Compose([
        transforms.Resize(300),
        transforms.CenterCrop(300),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])

In [ ]:
train_data = datasets.ImageFolder('./model_images/train', transform=train_transform)
val_data = datasets.ImageFolder('./model_images/valid', transform=test_transform)
#test_data = datasets.ImageFolder('../input/siim-isic-melanoma-classification/jpeg/test', transform=test_transform)

torch.manual_seed(42)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_data, batch_size=10, shuffle=True)

class_names = train_data.classes

print(class_names)
print(f'Training images available: {len(train_data)}')
print(f'Validation images available: {len(val_data)}')
#print(f'Testing images available:  {len(test_data)}')

In [ ]:
# Grab the first batch of 64 images
for images,labels in train_loader: 
    break


im = make_grid(images[:15], nrow=5)  # the default nrow is 8

# Inverse normalize the images
inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)
im_inv = inv_normalize(im)

# Print the images
plt.figure(figsize=(20,10))
plt.imshow(np.transpose(im_inv.numpy(), (1, 2, 0)));
#plt.imshow(im.permute(1,2,0))

In [ ]:
####Training Function#######

def train(n_epochs, train_loader, val_loader, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf
    losses=np.zeros((n_epochs,2))

    for epoch in range(n_epochs):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        # set the module to training mode
        model.train()
        for data, target in train_loader:
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda().unsqueeze(1).float()

            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out, target)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()

        ######################
        # validate the model #
        ######################
        # set the model to evaluation mode
        model.eval()
        for data, target in val_loader:
            with torch.no_grad():
                # move to GPU
                if use_cuda:
                    data, target = data.cuda(), target.cuda().unsqueeze(1).float()

                val_out = model(data)
                loss = criterion(val_out, target)
                valid_loss += loss.item()

        train_loss /= len(train_loader)
        valid_loss /= len(val_loader)
        losses[epoch]=(train_loss,valid_loss)

        # print training/validation statistics
        print(
            f'Epoch: {epoch+1} \tTraining Loss: {train_loss:.6f} \tValidation Loss: {valid_loss:.6f}')

        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ... Model saved ...')
            valid_loss_min = valid_loss
            torch.save(model.state_dict(), save_path)

    return model, losses

In [ ]:
use_cuda = torch.cuda.is_available()
use_cuda

In [ ]:
###Dounload Pre-trained Model

model = models.googlenet(pretrained=True)
model

In [ ]:
## Modify the classifier
torch.manual_seed(42)
model.fc = nn.Sequential(nn.Linear(1024, 128 , bias= False),
                                 nn.BatchNorm1d(128),
                                 nn.ReLU(),
                                 nn.Dropout(0.25),
                                 nn.Linear(128, 1),
                                 )
model

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
####TRAINING####
import time
start_time = time.time()
if use_cuda:
    model = model.cuda()

epochs = 1
model, losses = train(epochs,train_loader, val_loader, model, optimizer, criterion , use_cuda, 'model3.pt')
model.load_state_dict(torch.load('model3.pt'))

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

In [ ]:
torch.cuda.get_device_name(0)